# QFT Addition

**Companion notebook for:** [`add-with-qft.md`](./add-with-qft.md)

This notebook demonstrates:
- Adding numbers using QFT (in phase space)
- How phase rotations accumulate like binary addition
- Why this is elegant but fragile
- Comparing to classical addition

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QFT

np.set_printoptions(precision=3, suppress=True)

## 1. Addition via Phase Rotations

The basic idea: encode number → QFT → add via phase → inverse QFT.

In [2]:
def qft_add_constant(n_qubits, a, constant):
    """
    Add a constant to number 'a' using QFT.
    
    Args:
        n_qubits: Number of qubits
        a: Initial value (encoded in basis state)
        constant: Value to add
    """
    qc = QuantumCircuit(n_qubits)
    
    # Encode 'a' in basis state
    for bit_pos, bit in enumerate(format(a, f'0{n_qubits}b')[::-1]):
        if bit == '1':
            qc.x(bit_pos)
    
    # Apply QFT
    qc.append(QFT(n_qubits), range(n_qubits))
    
    # Add constant via phase rotations
    for qubit in range(n_qubits):
        angle = 2 * np.pi * constant * (2**qubit) / (2**n_qubits)
        qc.p(angle, qubit)
    
    # Inverse QFT to get answer
    qc.append(QFT(n_qubits, inverse=True), range(n_qubits))
    
    return qc

# Example: 3 + 2 = 5
n = 4  # Use 4 qubits
qc = qft_add_constant(n, a=3, constant=2)

state = Statevector(qc)
result = state.probabilities_dict()

print("Adding 3 + 2 using QFT:")
print("\nMost likely outcomes:")
sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)[:3]
for outcome, prob in sorted_results:
    print(f"  |{outcome}⟩ (decimal {int(outcome, 2)}): probability {prob:.4f}")

print("\n→ Result is |0101⟩ = |5⟩")
print("→ 3 + 2 = 5 ✓")

Adding 3 + 2 using QFT:

Most likely outcomes:
  |0101⟩ (decimal 5): probability 1.0000
  |1101⟩ (decimal 13): probability 0.0000
  |0110⟩ (decimal 6): probability 0.0000

→ Result is |0101⟩ = |5⟩
→ 3 + 2 = 5 ✓


/tmp/ipykernel_122917/4215275978.py:18: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_qubits), range(n_qubits))
/tmp/ipykernel_122917/4215275978.py:26: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_qubits, inverse=True), range(n_qubits))


## 2. How Phase Rotations Work

Each qubit gets rotated by an angle proportional to its significance.

In [3]:
# Show the angles for adding 2 to a 3-qubit register
n_qubits = 3
constant = 2

print(f"Adding {constant} to a {n_qubits}-qubit register:\n")
print("Qubit | Angle (radians) | Angle (degrees) | Fraction of 2π")
print("-" * 65)

for qubit in range(n_qubits):
    angle = 2 * np.pi * constant * (2**qubit) / (2**n_qubits)
    angle_deg = np.degrees(angle)
    fraction = angle / (2 * np.pi)
    print(f"  {qubit}   | {angle:14.4f} | {angle_deg:14.2f}° | {fraction:.4f}")

print("\n→ Less significant qubits get smaller rotations")
print("→ Rotations accumulate to represent the sum")

Adding 2 to a 3-qubit register:

Qubit | Angle (radians) | Angle (degrees) | Fraction of 2π
-----------------------------------------------------------------
  0   |         1.5708 |          90.00° | 0.2500
  1   |         3.1416 |         180.00° | 0.5000
  2   |         6.2832 |         360.00° | 1.0000

→ Less significant qubits get smaller rotations
→ Rotations accumulate to represent the sum


## 3. Testing Multiple Additions

In [4]:
# Test several additions
n_qubits = 4
test_cases = [(1, 1), (2, 3), (5, 4), (7, 8)]

print(f"Testing QFT addition with {n_qubits} qubits:\n")

for a, b in test_cases:
    qc = qft_add_constant(n_qubits, a, b)
    state = Statevector(qc)
    result = state.probabilities_dict()
    
    # Find most likely outcome
    best_outcome = max(result.items(), key=lambda x: x[1])
    result_decimal = int(best_outcome[0], 2)
    
    expected = (a + b) % (2**n_qubits)  # Modulo for overflow
    status = "✓" if result_decimal == expected else "✗"
    
    print(f"{a} + {b} = {result_decimal} (expected {expected}) {status}")

print("\n→ QFT addition works!")
print("→ But note: overflow wraps around (modular arithmetic)")

Testing QFT addition with 4 qubits:

1 + 1 = 2 (expected 2) ✓
2 + 3 = 5 (expected 5) ✓
5 + 4 = 9 (expected 9) ✓
7 + 8 = 15 (expected 15) ✓

→ QFT addition works!
→ But note: overflow wraps around (modular arithmetic)


/tmp/ipykernel_122917/4215275978.py:18: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_qubits), range(n_qubits))
/tmp/ipykernel_122917/4215275978.py:26: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_qubits, inverse=True), range(n_qubits))


## 4. Comparing to Classical Addition

Classical addition needs carry bits and full adders.  
QFT addition happens "in parallel" via phase.

In [5]:
def classical_add_circuit(n, a, b):
    """Classical ripple-carry adder for comparison"""
    # This would require many gates and ancilla qubits
    # Just showing the concept
    print("Classical addition circuit would need:")
    print(f"  • {n} input qubits for a")
    print(f"  • {n} input qubits for b")
    print(f"  • {n-1} ancilla qubits for carries")
    print(f"  • Multiple CNOT and Toffoli gates")
    print(f"  • Gates applied sequentially (carry ripples)")

def qft_add_circuit_summary(n):
    print("QFT addition circuit needs:")
    print(f"  • {n} input qubits")
    print(f"  • 0 ancilla qubits")
    print(f"  • QFT circuit (O(n²) gates)")
    print(f"  • {n} phase rotations")
    print(f"  • Inverse QFT circuit")
    print(f"  • All phase rotations happen in parallel")

n = 4
classical_add_circuit(n, 3, 2)
print()
qft_add_circuit_summary(n)

print("\n→ QFT: elegant, parallel, no ancillas")
print("→ Classical: sequential, needs extra qubits")
print("→ BUT: QFT is very sensitive to noise on hardware")

Classical addition circuit would need:
  • 4 input qubits for a
  • 4 input qubits for b
  • 3 ancilla qubits for carries
  • Multiple CNOT and Toffoli gates
  • Gates applied sequentially (carry ripples)

QFT addition circuit needs:
  • 4 input qubits
  • 0 ancilla qubits
  • QFT circuit (O(n²) gates)
  • 4 phase rotations
  • Inverse QFT circuit
  • All phase rotations happen in parallel

→ QFT: elegant, parallel, no ancillas
→ Classical: sequential, needs extra qubits
→ BUT: QFT is very sensitive to noise on hardware


## 5. Visualizing the Process

In [6]:
# Show states at each step
n = 3
a = 3
constant = 2

# Step 1: Encode 'a'
qc1 = QuantumCircuit(n)
for bit_pos, bit in enumerate(format(a, f'0{n}b')[::-1]):
    if bit == '1':
        qc1.x(bit_pos)
state1 = Statevector(qc1)

# Step 2: After QFT
qc2 = qc1.copy()
qc2.append(QFT(n), range(n))
state2 = Statevector(qc2)

# Step 3: After phase rotations
qc3 = qc2.copy()
for qubit in range(n):
    angle = 2 * np.pi * constant * (2**qubit) / (2**n)
    qc3.p(angle, qubit)
state3 = Statevector(qc3)

# Step 4: After inverse QFT
qc4 = qc3.copy()
qc4.append(QFT(n, inverse=True), range(n))
state4 = Statevector(qc4)

print("Evolution of 3 + 2 using QFT:\n")
print("Step 1 - Encode |3⟩:      ", state1.data)
print("Step 2 - After QFT:       ", state2.data)
print("Step 3 - After phase add: ", state3.data)
print("Step 4 - After QFT†:      ", state4.data)

result = state4.probabilities_dict()
print("\nFinal measurement would give:", max(result.items(), key=lambda x: x[1])[0])
print(f"Which is |{int(max(result.items(), key=lambda x: x[1])[0], 2)}⟩ = |5⟩")

Evolution of 3 + 2 using QFT:

Step 1 - Encode |3⟩:       [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Step 2 - After QFT:        [ 0.354+0.j    -0.25 +0.25j  -0.   -0.354j  0.25 +0.25j  -0.354+0.j
  0.25 -0.25j   0.   +0.354j -0.25 -0.25j ]
Step 3 - After phase add:  [ 0.354+0.j    -0.25 -0.25j   0.   +0.354j  0.25 -0.25j  -0.354+0.j
  0.25 +0.25j  -0.   -0.354j -0.25 +0.25j ]
Step 4 - After QFT†:       [ 0.+0.j  0.+0.j -0.+0.j -0.+0.j -0.+0.j  1.-0.j  0.+0.j  0.+0.j]

Final measurement would give: 101
Which is |5⟩ = |5⟩


/tmp/ipykernel_122917/1572619360.py:15: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc2.append(QFT(n), range(n))
/tmp/ipykernel_122917/1572619360.py:27: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc4.append(QFT(n, inverse=True), range(n))


## Summary

From this notebook, you should understand:

1. **QFT enables phase-based arithmetic** — addition becomes rotation
2. **No carry bits needed** — phase accumulates smoothly
3. **Modular arithmetic** — overflow wraps around naturally
4. **Elegant but fragile** — very sensitive to noise
5. **QFT as a tool** — organizes information for computation

This is why QFT matters:  
Not because addition itself is hard,  
but because **phase-based computation** enables algorithms like Shor's.

**Next:** [Algorithms](../06-algorithms/) — Where QFT becomes essential.